# OpenCV Simple example
going through some simple tests to make sure the camera works as expected
This is using the `opencv` library, install via pip with `opencv-python`


In [4]:
'''
Does the camera work correctly? 

Basically look for "opencv camera tutorial" on your platform and start from there.
'''
import cv2
import numpy as np


interface=1
camera = cv2.VideoCapture(interface)

while True:
    _, img = camera.read()
    if img is None:
        print(f"Unable to open camera {interface=}")
        break
    cv2.imshow("simple camera test", img)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()

In [5]:
# Part 0 - breaking up the image

while True: 
    _, img = camera.read()
    if img is None:
        print(f"Unable to open camera {interface=}")
        break
    # split image
    img_size = np.shape(img)[:-1]
    
    # Originally, this was showing channels in the channel colour
    # but it doesn't look great and doesn't explain the colour differences
    # Easily.
    # filler = (np.zeros((img_size)),np.zeros((img_size)))
    # b_img = cv2.convertScaleAbs(np.dstack((img[:,:,0],*filler)))
    # g_img = cv2.convertScaleAbs(np.dstack((filler[0],img[:,:,1],filler[0])))
    # r_img = cv2.convertScaleAbs(np.dstack((*filler,img[:,:,2])))
       
    # First, make an image, "convert" it to 3-channel so we have access to RGB values
    b_img = cv2.cvtColor(img[:,:,0],cv2.COLOR_GRAY2BGR)
    # Then superimpose text, in that colour value.
    b_img = cv2.putText(b_img,'Blue', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,0),2)
    g_img = cv2.cvtColor(img[:,:,1],cv2.COLOR_GRAY2BGR)
    g_img = cv2.putText(g_img,'Green', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),2)
    r_img = cv2.cvtColor(img[:,:,2],cv2.COLOR_GRAY2BGR)
    r_img = cv2.putText(r_img,'Red', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
    img = cv2.putText(img, 'Truth', (10,40), cv2.FONT_HERSHEY_COMPLEX,2,(0,0,0),2)

    
    # stack next to each other ( vstack / hstack )
    hza = np.hstack((b_img,g_img))
    hzb = np.hstack((r_img, img))
    hz = np.vstack((hza,hzb))
    cv2.imshow('3 Seperate channels', hz)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
del img, b_img, g_img, r_img, hz,hza,hzb

# Part 1 - Thresholding

```python
for pixel in image:
    if pixel.value > thresh:
        pixel = BLACK
    else:
        pixel = WHITE
```

In [9]:

BLUE_THRESHOLD_VALUE = 100
RED_THRESHOLD_VALUE = 150 
SIMPLE = True
THREE_PANEL = False
MASK_EXAMPLE = True
while True: 
    _, img = camera.read()
    if img is None:
        print(f"Unable to open camera {interface=}")
        break
       
    b_img = img[:,:,0]

    # this is a single channel image
    if SIMPLE:
        _,b_imgt = cv2.threshold(b_img, BLUE_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    else: 
        b_imgt = cv2.adaptiveThreshold(b_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5,2)
    #convert back to 3channel
    b_img = cv2.cvtColor(b_imgt, cv2.COLOR_GRAY2BGR)
    # Then superimpose text, in that colour value.
    b_img = cv2.putText(b_img,'BLUE THRESH', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,100,100),2)

    r_img = img[:,:,2]
    if SIMPLE:
        _,r_imgt = cv2.threshold(r_img, RED_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    else:
        r_imgt = cv2.adaptiveThreshold(r_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5,2)

    r_img = cv2.cvtColor(r_imgt, cv2.COLOR_GRAY2BGR)
    # Then superimpose text, in that colour value.
    r_img = cv2.putText(r_img,'RED THRESH', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(100,100,255),2)

    br_xor = np.bitwise_xor(b_imgt, r_imgt)
    if MASK_EXAMPLE:
        hz = cv2.putText(cv2.bitwise_and(img,img, mask = np.bitwise_xor(b_imgt,r_imgt)),'MASKED', (10,40), cv2.FONT_HERSHEY_COMPLEX,2,(255,255,255),2)
        

    elif THREE_PANEL:
        # stack next to each other ( vstack / hstack )
        hz = np.hstack((b_img,img,r_img))
        hzb = 0
    else:
        hz = np.hstack((b_img, r_img))
        hzb = np.hstack((img, cv2.putText(cv2.cvtColor(br_xor,cv2.COLOR_GRAY2BGR),
        'XOR', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,100,255),2)))

        hz = np.vstack((hz, hzb, 
        np.hstack((
            cv2.putText(
                cv2.cvtColor(
                    np.bitwise_and(b_imgt,br_xor),
                    cv2.COLOR_GRAY2BGR
                    ),'BLUE+XOR', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,200,200),2)
            ,
            cv2.putText(
                cv2.cvtColor(
                    np.bitwise_and(r_imgt, br_xor),
                    cv2.COLOR_GRAY2BGR
                    ),'RED+XOR', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(200,200,255),2)
        ))
        ))

    cv2.imshow('simple thresholding', hz)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
# del img, b_img, hzb, r_imgt, b_imgt, r_img, hz

# Part 2 Eroding and Dilating (Super simple, quick part)
This is just a very simple way to understand the erosion and dilation functions

In [10]:
KERNEL_SIZE = 5
while True: 
    _, img = camera.read()
    if img is None:
        print(f"Unable to open camera {interface=}")
        break
       
    b_img = img[:,:,0]

    _,b_imgt = cv2.threshold(b_img, BLUE_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    #convert back to 3channel
    b_img = cv2.cvtColor(b_imgt, cv2.COLOR_GRAY2BGR)
    # Then superimpose text, in that colour value.
    b_img = cv2.putText(b_img,'BLUE THRESH', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,100,100),2)

    r_img = img[:,:,2]
   
    _,r_imgt = cv2.threshold(r_img, RED_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
   
    r_img = cv2.cvtColor(r_imgt, cv2.COLOR_GRAY2BGR)
    # Then superimpose text, in that colour value.
    r_img = cv2.putText(r_img,'RED THRESH', (10,40),cv2.FONT_HERSHEY_COMPLEX,2,(100,100,255),2)

    mask = np.bitwise_xor(b_imgt,r_imgt)
    kernel = np.ones((KERNEL_SIZE, KERNEL_SIZE))
    eroded = cv2.erode(mask, kernel)
    dilate = cv2.dilate(mask,kernel)
    both = cv2.dilate(cv2.erode(mask,kernel),kernel)

    applied = cv2.bitwise_and(img,img, mask=both)

    hz = np.vstack((
        np.hstack((
            cv2.putText(cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR), "Original",(10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,100,100),2),
            cv2.putText(cv2.cvtColor(eroded,cv2.COLOR_GRAY2BGR), "Eroded",(10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,100,100),2)
            )),
        np.hstack((
            cv2.putText(cv2.cvtColor(dilate,cv2.COLOR_GRAY2BGR), "Dilate",(10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,100,100),2),
            cv2.putText(cv2.cvtColor(both,cv2.COLOR_GRAY2BGR), "Erode then Dilate",(10,40),cv2.FONT_HERSHEY_COMPLEX,2,(255,100,100),2)
            ))
    ))

    # draw simple lines to break up the image
    hz = cv2.line(hz, (640,0), (640,960), (100,100,255), 3)
    hz = cv2.line(hz, (0,480), (1280,480), (100,100,255),3)

    cv2.imshow('eroding', hz)
    cv2.imshow('truth', img)
    cv2.imshow('mask applied', applied)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
del hz, mask, kernel, eroded, dilate, both, applied 

# Anyway, on with BlobDector
Simple blob detector is an alorithim that just brings together "blobs" of pixel data, working off black and white images.


In [87]:
BLUE_THRESHOLD_VALUE = 120
RED_THRESHOLD_VALUE = 170
GREEN_THRESHOLD_VALUE = 120

KERNEL_SIZE = 3
kernel = np.ones((KERNEL_SIZE, KERNEL_SIZE))

# Contour Analysis
MIN_SIZE = 600
MAX_SIZE = 4000

while True: 
    _, img = camera.read()
    if img is None:
        print(f"Unable to open camera {interface=}")
        break

    b_img = img[:,:,0]
    img_size = np.shape(b_img)
    _,b_imgt = cv2.threshold(b_img, BLUE_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    b_imgt = cv2.dilate(cv2.erode(b_imgt,kernel),kernel)

    r_img = img[:,:,2]
    _,r_imgt = cv2.threshold(r_img, RED_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    r_imgt = cv2.dilate(cv2.erode(r_imgt,kernel),kernel)

    g_img = img[:,:,1]
    _,g_imgt = cv2.threshold(r_img, GREEN_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    g_imgt = cv2.dilate(cv2.erode(g_imgt,kernel),kernel)


    # b_imgt = np.bitwise_and(b_imgt, np.bitwise_or(r_imgt, g_imgt))
    # r_imgt = np.bitwise_and(r_imgt, np.bitwise_or(b_imgt, g_imgt))
    # g_imgt = np.bitwise_and(g_imgt, np.bitwise_or(r_imgt, b_imgt))

    # y_imgt = np.bitwise_and(g_imgt, r_imgt)
    # m_imgt = np.bitwise_and(b_imgt, r_imgt)
    # c_imgt = np.bitwise_and(b_imgt, g_imgt)

    # thresh_filter = np.bitwise_xor.reduce([r_imgt, b_imgt, g_imgt, m_imgt, y_imgt, c_imgt])
    thresh_filter = np.bitwise_and.reduce([np.bitwise_not(x) for x in [r_imgt,b_imgt,g_imgt]])
    

    contours, hierarchy = cv2.findContours(thresh_filter, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)    

    key_image = cv2.cvtColor(np.zeros(img_size,dtype=np.uint8),cv2.COLOR_GRAY2BGR)
    contour_image = img
  
    cdb = np.vstack((
        cv2.putText(cv2.cvtColor(b_imgt,cv2.COLOR_GRAY2BGR), "Blue",(10,40),cv2.FONT_HERSHEY_PLAIN,2,(255,100,100),2),
        cv2.putText(cv2.cvtColor(r_imgt,cv2.COLOR_GRAY2BGR), "Red",(10,40),cv2.FONT_HERSHEY_PLAIN,2,(100,100,255),2),
        cv2.putText(cv2.cvtColor(g_imgt,cv2.COLOR_GRAY2BGR), "Green",(10,40),cv2.FONT_HERSHEY_PLAIN,2,(100,255,100),2),
    ))

    for i in range(len(contours)):       
        cv2.drawContours(contour_image, contours, i, (100,255,100),2, cv2.LINE_AA, hierarchy,0)
        cv2.drawContours(key_image, contours, i, (100,255,100),1, cv2.LINE_AA, hierarchy,0)

    hz = np.hstack((img,cv2.cvtColor(thresh_filter, cv2.COLOR_GRAY2BGR)))
    hz = np.vstack((hz,
    np.hstack((key_image,contour_image))
    ))

    cv2.imshow('New Im', hz)
    cv2.imshow('Debug',cdb)

    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
# del img, b_img, hzb, r_imgt, b_imgt, r_img, hz

# Final Part ! 

In [88]:
BLUE_THRESHOLD_VALUE = 120
RED_THRESHOLD_VALUE = 170
GREEN_THRESHOLD_VALUE = 120

KERNEL_SIZE = 3
kernel = np.ones((KERNEL_SIZE, KERNEL_SIZE))

# Contour Analysis
MIN_SIZE = 600
MAX_SIZE = 4000


while True: 
    _, img = camera.read()
    if img is None:
        print(f"Unable to open camera {interface=}")
        break


    width = int(img.shape[1] * 2)
    height = int(img.shape[0] * 2)
    dim = (width, height)

    # resize image
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    b_img = img[:,:,0]
    img_size = np.shape(b_img)
    _,b_imgt = cv2.threshold(b_img, BLUE_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    b_imgt = cv2.dilate(cv2.erode(b_imgt,kernel),kernel)

    r_img = img[:,:,2]
    _,r_imgt = cv2.threshold(r_img, RED_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    r_imgt = cv2.dilate(cv2.erode(r_imgt,kernel),kernel)

    g_img = img[:,:,1]
    _,g_imgt = cv2.threshold(r_img, GREEN_THRESHOLD_VALUE,255, cv2.THRESH_BINARY)
    g_imgt = cv2.dilate(cv2.erode(g_imgt,kernel),kernel)
    thresh_filter = np.bitwise_and.reduce([np.bitwise_not(x) for x in [r_imgt,b_imgt,g_imgt]])
    
    contours, hierarchy = cv2.findContours(thresh_filter, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)    

  
    for i in range(len(contours)):
        c = contours[i]
        area = cv2.contourArea(c)
        if area < 3000:
            continue
        M = cv2.moments(c)
        try:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        except ZeroDivisionError:
            continue

        cv2.drawContours(img, contours, i, (100,255,100),2, cv2.LINE_AA, hierarchy,0)
        
        if area > 40_000:
            size= "Big"
        else:
            size = "Small"

        perimeter = cv2.arcLength(c,True)
        approx = cv2.approxPolyDP(c, 0.04 * perimeter, True)

        if len(approx) == 3:
            shape = "Triangle"
        elif len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            shape = "Square" if abs(ar-1) < .20 else "Rectangle"
        else:
            shape = "Circle"

        img = cv2.putText(img,f"{size} {shape}", (cX-100,cY), cv2.FONT_HERSHEY_PLAIN, 2, (255,100,255),2)



    cv2.imshow('Final Image', img)

    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()